-In test data set we didn't have some columns so first we need predict the columns values based on available values.

In [54]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Library for Modelling 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

#Library for Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%matplotlib inline

In [55]:
train_data=pd.read_csv('competitive-data-science-predict-future-sales/sales_train.csv')
test_data=pd.read_csv('competitive-data-science-predict-future-sales/test.csv')

In [56]:
train_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [57]:
#Changing Negative numbers into Positive
train_data['item_price']=train_data['item_price'].abs()
train_data['item_cnt_day']=train_data['item_cnt_day'].abs()

In [58]:
#Checking if the training data has any Null values or not
null_percent=100*(train_data.isnull().sum()/len(train_data))
print(null_percent)

date              0.0
date_block_num    0.0
shop_id           0.0
item_id           0.0
item_price        0.0
item_cnt_day      0.0
dtype: float64


In [59]:
#Grouping item price and item count per month
train_data=train_data.groupby(['date_block_num','shop_id','item_id']).agg({'item_price':'last','item_cnt_day':'sum'}).reset_index()

#Changing the name of the item_cnt_day to item_cnt_month
train_data=train_data.rename(columns={'item_cnt_day':'item_cnt_month'})
train_data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,0,0,32,221.0,6.0
1,0,0,33,347.0,3.0
2,0,0,35,247.0,1.0
3,0,0,43,221.0,1.0
4,0,0,51,127.0,2.0


In [60]:
for feature in train_data.columns:
    if 0 in train_data[feature].unique():
        pass
    else:
        print(feature)
        train_data[feature]=np.log(train_data[feature])

item_price
item_cnt_month


In [61]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(train_data)

train_data = pd.DataFrame(scaler.transform(train_data), columns = train_data.columns)

train_data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,-1.536816,-1.983784,-1.706875,-0.649855,2.059322
1,-1.536816,-1.983784,-1.706715,-0.213787,1.032998
2,-1.536816,-1.983784,-1.706394,-0.542350,-0.593688
3,-1.536816,-1.983784,-1.705112,-0.649855,-0.593688
4,-1.536816,-1.983784,-1.703830,-1.185296,0.432636


In [62]:
x_train = train_data.drop(labels=["item_cnt_month", "date_block_num", "item_price"], axis=1)

y_train = train_data["date_block_num"]

In [63]:
scaler.fit(test_data)

test_data = pd.DataFrame(scaler.transform(test_data), columns = test_data.columns)

test_data.isnull().sum()

ID         0
shop_id    0
item_id    0
dtype: int64

In [64]:
x_test = test_data.drop(labels=["ID"], axis=1)

In [65]:
x_train.head()

,shop_id,item_id
0,-1.983784,-1.706875
1,-1.983784,-1.706715
2,-1.983784,-1.706394
3,-1.983784,-1.705112
4,-1.983784,-1.703830


In [66]:
y_train.head()

0   -1.536816
1   -1.536816
2   -1.536816
3   -1.536816
4   -1.536816
Name: date_block_num, dtype: float64

In [67]:
x_test.head()

,shop_id,item_id
0,-1.517083,-0.956781
1,-1.517083,-0.911520
2,-1.517083,-0.925434
3,-1.517083,-0.925594
4,-1.517083,-0.919837


In [68]:
# linear_prediction.min(), linear_prediction.max()

In [69]:
# rr_model=RandomForestRegressor(n_estimators=5)
# rr_model.fit(x_train,y_train)

# rr_predict=rr_model.predict(x_test)
# print(rr_predict)

In [70]:
#Training the model
linear_model=LinearRegression()
linear_model.fit(x_train,y_train)

#Testing the model
linear_prediction=linear_model.predict(x_test)

#output is the predicted cnt_per_month
print(linear_prediction)

[-0.0277896  -0.02809375 -0.02800025 ...  0.01206375  0.00788194
  0.02795696]


In [71]:
val = pd.Series(rr_predict)

In [72]:
val.to_csv("predicted/date_block_num.csv", header = ["predicted_date_block_num"], index=False)